In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
from matplotlib.pyplot import *

In [2]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=5, bidir=True)


        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                   
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])  

        coeff_tr = []
        biases_tr = []   

        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        #print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [6]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [21]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

0.9499969482421875

# Changing # of internal units
Using 50% of the dataset, 10 features, 5 internal units

In [ ]:
datasets = ["Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv", "Friday-16-02-2018_TrafficForML_CICFlowMeter.csv"]

features_Th15022018 = ['Fwd Seg Size Min', 'Init Fwd Win Byts', 'Bwd Pkt Len Max', 'Fwd IAT Min', 'Bwd IAT Mean', 'Fwd IAT Tot', 'Flow IAT Mean', 'Flow Duration', 'Fwd IAT Mean', 'Pkt Len Max', 
                       'Fwd IAT Max', 'Bwd Pkt Len Std', 'PSH Flag Cnt', 'Flow IAT Min', 'Bwd IAT Max', 'ACK Flag Cnt', 'Flow IAT Max', 'Idle Max', 'Fwd Header Len', 'Flow IAT Std']
features_Fr16022018 = ['Fwd Pkt Len Std', 'Pkt Len Std', 'Fwd Seg Size Avg', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Max', 'TotLen Fwd Pkts', 'Pkt Len Var', 'Bwd Pkt Len Max', 'Pkt Len Max', 'Pkt Size Avg',
                       'Bwd Seg Size Avg', 'Flow IAT Std', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Fwd IAT Max', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Fwd IAT Std', 'Subflow Fwd Byts', 'Flow IAT Mean']

numFeatures = [10, 15, 20]
fracOfData = [0.5, 0.75, 1]
numInternalUnits = [5, 10, 20]

In [3]:
dataset = "Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv"
df = pd.read_csv(dataset)
num_features = 10 # should be 10, 15, or 20
print(str(num_features) + " features")
features = features_Th15022018[0:num_features]

In [3]:
for fraction in fracOfData:
    print("fraction:" + str(fraction))
    data = df.sample(frac=fraction, replace=True, random_state=1)
            
    # get X and y. Normalize X and make it into 3D shape for reservoir
    num_col = data.shape[1]
    num_row = data.shape[0]

    X_data = data[features]
    X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(X_data.values)
    X = np.nan_to_num(x_scaled)
    if len(X.shape) < 3:
        X = np.atleast_3d(X)
    y = data['Label'].apply(targetify)
    print("Finished loading X and y......")

    # split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

    print("X_train shape:" + str(X_train.shape), "y_train shape:" + str(y_train.shape))
    print("X_test shape:" + str(X_test.shape), "y_test shape:" + str(y_test.shape))

    pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
    ridge_embedding = Ridge(alpha=10, fit_intercept=True)
    readout = Ridge(alpha=5)

    for n in numInternalUnits:
        print(str(n) + "internal units")

        #run through reservoir
        res = Reservoir(n_internal_units=n, spectral_radius=0.6, leak=0.6,
             connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)
        input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
        print("Finished loading training reservoir embedding......")
        input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
        print("Finished loading testing reservoir embedding......")

        #fit output
        readout.fit(input_repr, y_train)
        pred_class = readout.predict(input_repr_te)
        predictions = [int(round(x)) for x in pred_class]
        true_class = list(y_test)

        #analysis
        compdf = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
        compdf = compdf.sort_values('pred_class', ascending=False)
        compdf.to_csv(dataset.split('_')[0] + '_' + str(fraction) + '_' + str(num_features) + '_' + str(n) + '.csv')
        accuracy = np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

        print("# of nonzero:" np.count_nonzero(predictions))
        print("accuracy" + accuracy)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
128037,53,17,15/02/2018 11:18:07,2261,1,1,49,164,49,49,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491755,3389,6,15/02/2018 01:57:33,2837721,9,7,1144,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
470924,3389,6,15/02/2018 09:21:14,1645626,8,7,1148,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
791624,3389,6,15/02/2018 02:46:24,2486564,8,8,1144,1874,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491263,53,17,15/02/2018 12:34:09,1758,1,1,30,207,30,30,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
